In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import tensorflow as tf
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['dnn-model', 'new-york-city-taxi-fare-prediction', 'submisiion-nyc-taxi']


In [2]:
os.listdir("../input/dnn-model")

['model_.ckpt.meta',
 'model_.ckpt.index',
 'model_.ckpt.data-00000-of-00001',
 'checkpoint']

In [3]:
df_test=pd.read_csv("../input/new-york-city-taxi-fare-prediction/test.csv")

In [4]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


Functions used

In [5]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    #print(targ_pre)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear','hour',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
        
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)
        
###Harv Distance        
def distance( data):

    radius = 6371 # km
    lon1=data[:,0]
    lat1=data[:,1]
    lon2=data[:,2]
    lat2=data[:,3]
    #print(lat2-lat1)
    dlat = np.radians(lat2-lat1)
    dlon = np.radians(lon2-lon1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) \
            * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan(np.sqrt(a), np.sqrt(1-a))
    d = radius * c

    return d

In [6]:
he_init = tf.contrib.layers.variance_scaling_initializer()

def dnn(inputs,training, n_hidden_layers=8, name=None,
        activation=tf.nn.relu, initializer=he_init):
    n_neurons=[2000,1000,500,250,125,50,25,10]
 
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            
            
       
            inputs = tf.layers.dense(inputs, n_neurons[layer],
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        

            inputs=tf.nn.relu(inputs, name="hidden%d_out" % (layer + 1))
        
        return inputs

In [7]:
df_test['Herv_Dist'] =distance(np.float64(df_test.values[:,2:6]))

In [8]:
add_datepart(df_test, 'pickup_datetime', drop=True)

In [9]:
feature_cols=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'pickup_datetimeYear',
       'pickup_datetimeMonth', 'pickup_datetimeWeek', 'pickup_datetimeDay',
       'pickup_datetimeDayofweek', 'pickup_datetimeDayofyear',
       'pickup_datetimehour', 'pickup_datetimeElapsed', 'Herv_Dist']

In [10]:
df_test[feature_cols].head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimehour,pickup_datetimeElapsed,Herv_Dist
0,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,5,27,1,27,13,1422364104,2.323260
1,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,5,27,1,27,13,1422364104,2.425353
2,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,40,8,5,281,11,1318074824,0.618628
3,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,48,1,5,336,21,1354396332,1.961032
4,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,48,1,5,336,21,1354396332,5.387301


In [11]:
##Mean and sigma for scaling
mu=np.array([ -7.39752352e+01,   4.07510864e+01,  -7.39743620e+01,
         4.07514412e+01,   1.69111912e+00,   2.01173779e+03,
         6.26937910e+00,   2.54649417e+01,   1.57119467e+01,
         3.04109087e+00,   1.75307310e+02,   1.35101716e+01,
         1.33224990e+09,   3.34143936e+00])
sigma=np.array([  4.26467712e-02,   3.18110081e-02,   4.13962939e-02,
         3.48417371e-02,   1.30694141e+00,   1.86550121e+00,
         3.43641982e+00,   1.49473195e+01,   8.68516050e+00,
         1.94912410e+00,   1.04798866e+02,   6.51677611e+00,
         5.84916113e+07,   4.08371701e+00])

In [12]:
x_test_unscl=df_test[feature_cols].values

In [13]:
x_test=(x_test_unscl-mu)/sigma

DNN Network

In [14]:
tf.reset_default_graph()
n_inputs=14
n_outputs=1

In [15]:
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name='y')
training=tf.placeholder_with_default(False,shape=(),name='training')

In [16]:
with tf.name_scope("dnn"):
    y_pred=tf.layers.dense(dnn(X,training),n_outputs,name="Outputs",kernel_initializer=he_init,activation=tf.nn.relu)

In [17]:
with tf.name_scope("loss"):

    error=y_pred-y
    mse=tf.reduce_mean(tf.square(error),name='MSE')

In [18]:
l_rate=0.0001
with tf.name_scope("train"):

    optimizer=tf.train.AdamOptimizer(learning_rate=l_rate)
    t_op=optimizer.minimize(mse)

In [19]:
init=tf.global_variables_initializer()
saver=tf.train.Saver()

In [20]:
sess=tf.InteractiveSession()
sess.run(init)

In [21]:
saver.restore(sess,'../input/dnn-model/model_.ckpt')

INFO:tensorflow:Restoring parameters from ../input/dnn-model/model_.ckpt


In [22]:
y_test=y_pred.eval(feed_dict={X:x_test})

In [23]:
sess.close()

In [24]:
my_submission =pd.DataFrame(np.concatenate((df_test['key'].values.reshape(-1,1),y_test),axis=1),columns=['key','fare_amount'])

In [25]:
my_submission.to_csv('submission.csv', index=False)

In [26]:
y_test

array([[ 9.073902],
       [10.794902],
       [ 5.012144],
       ...,
       [55.461636],
       [21.122755],
       [ 7.738399]], dtype=float32)